In [1]:
# test_imports in this cell, restart kernel whenever edits to modules are made

import sys
import os

# Append the src directory to the path
current_dir = os.getcwd()
src_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))
if src_dir not in sys.path:
    sys.path.append(src_dir)

#import config based on which machine the jupyter notebook is running on
from configs.config_local import *
from preprocessing import *
from hub_calls import *
from postprocessing import *
from utils import *

def test_imports():
    print("testing config class")
    config = Config()
    print(config)

    print("testing 2d contacts preprocessing classes")
    query = HiCQuery(config)
    print(query)

    print("testing helper function imports")
    out = ValidLoop(config, 10000)
    print(out)
    
    #other prints for imported classes

test_imports()

testing config class
testing 2d contacts preprocessing classes
HiC file loaded
testing helper function imports


In [4]:
####### Loading Config object #######

config = Config()
query = HiCQuery(config)

# check if hic file is readable
inform = query.read_hic_header(config.paths.hic_file)
print(inform)

#set parameters
current_chrom = config.genomic_params.chromosomes[0]
current_res = config.genomic_params.resolutions[0]
currest_res0 = config.genomic_params.res_strs[0]

HiC file loaded
{'version': '8', 'master_index': '3126283484', 'genome_id': '/var/lib/cwl/stga3f80174-ba8e-4d25-8491-09d0707bbd92/4DNFI823LSII.chrom.sizes', 'attributes': {b's': b'o'}}


In [ ]:
####### Thresholding the oe matrix for edgelist storage #######
hic = query.hic #create hic straw object
